In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import sys
import torch

In [ ]:
sys.path.append("..")
plt.rcParams["figure.figsize"] = (3, 2)
mpl.rcParams["figure.dpi"] = 300

# Very simple example from the BoTorch docs using `easyBO`

See [here](https://botorch.org/v/0.1.0/tutorials/fit_model_with_torch_optimizer). In this simple example, we do the following:
1. Initialize a single task GP regressor from dummy training data
2. Assume homoscedastic noise
3. Train the GP hyperparameters
4. Plot the results

In [ ]:
from easybo.gp import EasySingleTaskGPRegressor
from easybo.bo import ask
from easybo.logger import logging_mode, logger
from easybo.utils import plot_1d_fit, get_dummy_1d_sinusoidal_data, set_grids

**Step 1:** create some dummy training data.

In [ ]:
grid, train_x, train_y = get_dummy_1d_sinusoidal_data()

In [ ]:
train_y = train_y * 100
train_x = train_x * 10
grid = grid * 10

**Step 2:** get the initial model conditioned on the training data, and run inference on the un-optimized GP, just to see what it looks like.

In [ ]:
model = EasySingleTaskGPRegressor(
    train_x=train_x,
    train_y=train_y,
    normalize_inputs_to_unity=True,
    standardize_outputs=True,
)

**Step 3:** training: optimize the hyper-parameters (by default, this is just a kernel of the form `Const x RBF`. We can optionally use the `mode` context manager to indicate the logging level of the procedure. Note that this context manager can be used with any function, class, method, etc. in `easybo`.

In [ ]:
with logging_mode(debug=True, debug_simple=True):
    model.train_()

In [ ]:
preds = model.predict(grid=grid)

**Step 4:** now we can experiment with adding more data. During e.g. autonomous experimentation, we might want to add new data to the model, and condition it appropriately. The `tell` API calls BoTorch's `condition_on_observations` method, handing necessary transforms along the way.

In [ ]:
new_x = np.array([2.25, 2.50]).reshape(-1, 1) * 10
new_y = np.array([1, 2]).reshape(-1, 1) * 100
with logging_mode(debug=True, debug_simple=True):
    new_model = model.tell(new_x=new_x, new_y=new_y, retrain=False)

**Step 5:** plot the results!

In [ ]:
with logging_mode(debug=True, debug_simple=True):
    new_model.train_()

In [ ]:
ground_truth = new_model.dream(seed=123)
sampled = ground_truth.predict(grid=grid)["mean"]

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(3, 2), sharey=True, sharex=True)

ax = axs[0]
set_grids(ax, grid=False)
plot_1d_fit(ax=ax, model=model, grid=grid * 2)
ax.set_ylabel(r"$f(x)$")

ax = axs[1]
set_grids(ax, grid=False)
plot_1d_fit(ax=ax, model=new_model, grid=grid * 2)
ax.legend(bbox_to_anchor=(1.05, 0.5), loc="center left", frameon=False)

ax.plot(grid, sampled, linewidth=1)

ax = fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor="none", top="off", bottom="off", left="off", right="off")
ax.set_xticks([])
ax.set_yticks([])
ax.set_xlabel(r"$x$", labelpad=15)

plt.subplots_adjust(wspace=0.1)
plt.show()
# plt.savefig("figure1.png", dpi=300, bbox_inches="tight")